<a href="https://colab.research.google.com/github/vishnubanna/stocktrader/blob/master/Stock_trend_prediction_NURAL_NET_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow==2.0.0-alpha0

In [0]:
import pandas as pd

import time
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn import preprocessing, svm
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression


import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

In [0]:
def get_data(ticker):
  try:
    print(ticker)
    #start = dt.datetime(1950, 1, 1)
    start = dt.datetime(2000, 1, 1)
    #start = dt.datetime(2010, 1, 1)
    end = dt.datetime.today() - dt.timedelta(days = 1)
    df = pdr.get_data_yahoo(ticker, start, end)
    
    while emptychek(df):
        df = pdr.get_data_yahoo(ticker, start, end)
    
#     print((df.dropna(inplace = True)).empty)
    
#     while df.empty: 
#         print('here')
#         df = pdr.get_data_yahoo(ticker, start, end)
        
#     empty = True
#     if empty: 
#         try: 
#             print(df.head())
#             empty = False
#         except:
#             empty = True 
    
#     print(empty)
        
    print(df.head())
    print(df.tail())
    return(df)
  except:
    pass

def emptychek(df):
    empty = True
    if empty: 
        try: 
            print(df.head())
            empty = False
        except:
            empty = True 
    
    print(empty)
    return
  
def graphData(stock):
    try:
        df = stock
        df.reset_index(inplace = True)
        fig = plt.figure()
        ax1 = plt.subplot(2,1,1)
        ax1.plot(df.Date, df.Open)
        ax1.plot(df.Date, df.High)
        ax1.plot(df.Date, df.Low)
        ax1.plot(df.Date, df.Close)
        ax1.grid(True)
    
        ax2 = plt.subplot(2,1,2)
        ax2.bar(df.Date,df.Volume)
    
        ax1.xaxis.set_major_locator(mticker.MaxNLocator(10))
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    
        for label in ax1.xaxis.get_tickerlabels():
            label.set_rotation(45)     
        return
    except:
        pass

def timeSeries(df):
    df['f01'] = df['Adj Close'].shift(-1)
    df['f02'] = df['Adj Close'].shift(-2)
    df['f03'] = df['Adj Close'].shift(-3)
    df['f04'] = df['Adj Close'].shift(-4)
    df['f05'] = df['Adj Close'].shift(-5)
    df['f06'] = df['Adj Close'].shift(-6)
    df['f07'] = df['Adj Close'].shift(-7)
    
    df = df[['Close', 'f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07']]
    
    print(df.head())
    print(df.tail())
    df.dropna(inplace = True)
    
    return df

def dataSpecify(df):
    print(df.head(), df.tail())
    
    df['50 MAvg'] = df['Close'].rolling(window = 50, min_periods = 0).mean()
    df['20 MAvg'] = df['Close'].rolling(window = 20, min_periods = 0).mean()
    df['10 MAvg'] = df['Close'].rolling(window = 10, min_periods = 0).mean()
    df['VolumeMAvg'] = df['Volume'].rolling(window = 50, min_periods = 0).mean()
    df['volitility1'] = (df['Close']-df['50 MAvg'])/df['Close'] * 100

    df['vshift'] = df['volitility1'].shift(1)
    df['volSlope'] = df['volitility1'] - df['vshift']

    df['50MAshift'] = df['50 MAvg'].shift(1)
    df['50MASlope'] = (df['50 MAvg'] - df['50MAshift'])

    df['% change day'] = (df['Open'] - df['Close']) / df['Open'] * 100
    df['% daily volit'] = (df['High'] - df['Close']) / df['High'] * 100
    
    df['cShift'] = df['Close'].shift(1)
    #
    df['%volit'] = (df['cShift'] - df['Close']) / df['cShift'] * 100
    
    df.fillna(0, inplace = True)
    
    
    print(df['%volit'].head(30))
    
    
    #df = df[['Close', '50 MAvg', 'Volume', 'volitility1', '% daily volit','% change day' ,'VolumeMAvg', '%volit']]
    #df = df[['Close', '50 MAvg', 'Volume', 'volitility1', '% daily volit','% change day' ,'VolumeMAvg', '%volit']]
    df = df[['Close', 'Open', '50 MAvg', 'Volume', 'volitility1', '% daily volit','% change day' ,'VolumeMAvg', '%volit']]
    print(df.head(), df.tail())
    
    return df

def certainty_testing(df):
    close = np.ndarray.flatten(df.VolumeMAvg.values)
    test = (df.Volume.values).flatten('F')[:len(close)]

    print(test, len(test))
    print(len(close))

    certainty = []

    for i in range(len(close)):
        volume = test[i]
        vavg = close[i]

        if volume > vavg:
            certainty.append(1)
        else:
            certainty.append(0)
    
    
    print(volume)
    print(test[1249])
    print(vavg)
    print(close[1249])
    print(certainty)
    
    return certainty

def train_test_sets(df, scaling = False, days = 1):
    forecast_col = 'Close'
    df.fillna(-99999, inplace = True)
    forecast_out = days

    df['Label'] = df[forecast_col].shift(-forecast_out)
    print(df.tail())

    #X = np.array(df.drop(['Label', 'Close'], axis = 1))
    X = np.array(df.drop(['Label', '%volit'], axis = 1))
    if scaling:
        X = preprocessing.scale(X)
    X_lately = X[-forecast_out:] #forcastout = 30, (-)30 from the bottom 
    X = X[:-forecast_out]

    df.dropna(inplace = True)
    y = np.array(df['Label'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

    print(X_train)
    
    return X_train, X_test, y_train, y_test, X_lately




In [0]:
df = get_data('AAPL')

AAPL
[*********************100%***********************]  1 of 1 downloaded
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2000-01-03  3.745536  4.017857  3.631696  3.997768   2.655498  133949200
2000-01-04  3.866071  3.950893  3.613839  3.660714   2.431611  128094400
2000-01-05  3.705357  3.948661  3.678571  3.714286   2.467196  194580400
2000-01-06  3.790179  3.821429  3.392857  3.392857   2.253689  191993200
2000-01-07  3.446429  3.607143  3.410714  3.553571   2.360442  115183600
False
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2000-01-03  3.745536  4.017857  3.631696  3.997768   2.655498  133949200
2000-01-04  3.866071  3.950893  3.613839  3.660714   2.431611  128094400
2000-01-05  3.705357  3.948661  3.678571  3.714286   2.467196  194580400
2000-01-06  3.790179  3.821429  3.392857  3

In [0]:
data = dataSpecify(df)

                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2000-01-03  3.745536  4.017857  3.631696  3.997768   2.655498  133949200
2000-01-04  3.866071  3.950893  3.613839  3.660714   2.431611  128094400
2000-01-05  3.705357  3.948661  3.678571  3.714286   2.467196  194580400
2000-01-06  3.790179  3.821429  3.392857  3.392857   2.253689  191993200
2000-01-07  3.446429  3.607143  3.410714  3.553571   2.360442  115183600                   Open        High  ...   Adj Close    Volume
Date                                ...                      
2019-05-22  184.660004  185.710007  ...  182.779999  29748600
2019-05-23  179.800003  180.539993  ...  179.660004  36529700
2019-05-24  180.199997  182.139999  ...  178.970001  23714700
2019-05-28  178.919998  180.589996  ...  178.229996  27948200
2019-05-29  176.419998  179.350006  ...  177.380005  28452000

[5 rows x 6 columns]
Date
2000-01-03     0.000000
2000

In [0]:
ts = timeSeries(df)

               Close       f01       f02  ...       f05       f06       f07
Date                                      ...                              
2000-01-03  3.997768  2.431611  2.467196  ...  2.318927  2.200312  2.068352
2000-01-04  3.660714  2.467196  2.253689  ...  2.200312  2.068352  2.295204
2000-01-05  3.714286  2.253689  2.360442  ...  2.068352  2.295204  2.382682
2000-01-06  3.392857  2.360442  2.318927  ...  2.295204  2.382682  2.465714
2000-01-07  3.553571  2.318927  2.200312  ...  2.382682  2.465714  2.527986

[5 rows x 8 columns]
                 Close         f01         f02  ...  f05  f06  f07
Date                                            ...               
2019-05-22  182.779999  179.660004  178.970001  ...  NaN  NaN  NaN
2019-05-23  179.660004  178.970001  178.229996  ...  NaN  NaN  NaN
2019-05-24  178.970001  178.229996  177.380005  ...  NaN  NaN  NaN
2019-05-28  178.229996  177.380005         NaN  ...  NaN  NaN  NaN
2019-05-29  177.380005         NaN         N

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
#certainty_t = certainty_testing(df)
#ct = pd.DataFrame(certainty_t)

#X_train, X_test, y_train, y_test, X_lately = train_test_sets(ts, scaling = False, days = 8)
X_train, X_test, y_train, y_test, X_lately = train_test_sets(data, scaling = True, days = 1)
print(X_lately)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


                 Close        Open   50 MAvg  ...  VolumeMAvg    %volit       Label
Date                                          ...                                  
2019-05-22  182.779999  184.660004  195.8866  ...  30489506.0  2.047163  179.660004
2019-05-23  179.660004  179.800003  195.8456  ...  30599450.0  1.706967  178.970001
2019-05-24  178.970001  180.199997  195.7504  ...  30602154.0  0.384060  178.229996
2019-05-28  178.229996  178.919998  195.5926  ...  30380260.0  0.413480  177.380005
2019-05-29  177.380005  176.419998  195.3798  ...  30424904.0  0.476907         NaN

[5 rows x 10 columns]
[[-0.41282348 -0.4051654  -0.42406739 ...  0.25226403  0.71515084
   0.04810571]
 [-0.85619817 -0.85592525 -0.86031522 ...  0.11117283  0.34999178
   0.04180481]
 [ 0.87950251  0.91404276  1.03279991 ...  0.37089152  0.89456945
  -1.04093769]
 ...
 [-0.83222271 -0.83348103 -0.83241628 ... -0.18240173 -0.47774235
   0.81677955]
 [-0.32325805 -0.32051544 -0.32154829 ...  0.27836041  0.232

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
import tensorflow as tf
import tensorflow.keras as ks



layers = [
    ks.layers.InputLayer(input_shape = (len(X_train[1]), )),
    ks.layers.Dense(units = 64, activation = ks.activations.relu),
    #ks.layers.Dropout(0.25),
    ks.layers.Dense(units = 64, activation = ks.activations.relu),
    #ks.layers.Dropout(0.25),
    ks.layers.Dense(units = 1)
]
model = ks.Sequential(layers)

model.compile(optimizer = ks.optimizers.Adam(),
              loss = ks.losses.MeanSquaredError(),
              metrics = [ks.metrics.MeanAbsoluteError(), ks.metrics.MeanSquaredError()]
)

model.fit(x = X_train, y = y_train, batch_size = 20, epochs = 100)
model.evaluate(x = X_test, y = y_test, batch_size = 1)

prediction = model.predict(X_lately, verbose = 0)

print(prediction)


Epoch 1/100
3904/3904 [==============================] - 0s 98us/sample - loss: 3248.3356 - mean_absolute_error: 35.4359 - mean_squared_error: 3248.3342
Epoch 2/100
3904/3904 [==============================] - 0s 65us/sample - loss: 113.9717 - mean_absolute_error: 7.8288 - mean_squared_error: 113.9716
Epoch 3/100
3904/3904 [==============================] - 0s 65us/sample - loss: 50.6821 - mean_absolute_error: 5.3881 - mean_squared_error: 50.6821
Epoch 4/100
3904/3904 [==============================] - 0s 64us/sample - loss: 24.0032 - mean_absolute_error: 3.6796 - mean_squared_error: 24.0032
Epoch 5/100
3904/3904 [==============================] - 0s 65us/sample - loss: 12.5375 - mean_absolute_error: 2.6064 - mean_squared_error: 12.5375
Epoch 6/100
3904/3904 [==============================] - 0s 66us/sample - loss: 7.7419 - mean_absolute_error: 2.0164 - mean_squared_error: 7.7419
Epoch 7/100
3904/3904 [==============================] - 0s 64us/sample - loss: 5.5647 - mean_absolute_erro

In [0]:
import tensorflow as tf
import tensorflow.keras as ks



layers = [
    ks.layers.InputLayer(input_shape = (len(X_train[1]), )),
    ks.layers.Dense(units = 128, activation = ks.activations.relu),
    #ks.layers.Dropout(0.25),
    ks.layers.Dense(units = 64, activation = ks.activations.relu),
    #ks.layers.Dropout(0.25),
    ks.layers.Dense(units = 1)
]
model2 = ks.Sequential(layers)

model2.compile(optimizer = ks.optimizers.Adam(),
              loss = ks.losses.MeanSquaredError(),
              metrics = [ks.metrics.MeanAbsoluteError(), ks.metrics.MeanSquaredError()]
)

model2.fit(x = X_train, y = y_train, batch_size = 20, epochs = 100)
model2.evaluate(x = X_test, y = y_test, batch_size = 1)

prediction1 = model2.predict(X_lately, verbose = 0)

print(prediction1)

Epoch 1/100
3904/3904 [==============================] - 0s 84us/sample - loss: 2619.2587 - mean_absolute_error: 29.9986 - mean_squared_error: 2619.2581
Epoch 2/100
3904/3904 [==============================] - 0s 66us/sample - loss: 87.0546 - mean_absolute_error: 6.9298 - mean_squared_error: 87.0546
Epoch 3/100
3904/3904 [==============================] - 0s 69us/sample - loss: 30.6904 - mean_absolute_error: 4.1713 - mean_squared_error: 30.6904
Epoch 4/100
3904/3904 [==============================] - 0s 65us/sample - loss: 13.2954 - mean_absolute_error: 2.7019 - mean_squared_error: 13.2954
Epoch 5/100
3904/3904 [==============================] - 0s 69us/sample - loss: 7.5503 - mean_absolute_error: 2.0015 - mean_squared_error: 7.5503
Epoch 6/100
3904/3904 [==============================] - 0s 66us/sample - loss: 4.6522 - mean_absolute_error: 1.5367 - mean_squared_error: 4.6522
Epoch 7/100
3904/3904 [==============================] - 0s 64us/sample - loss: 3.4853 - mean_absolute_error: 1

In [0]:
'''
AAPL
today close:[[179.81609]] or [[181.13953]] (+/- 2) actual 178.97

tomorrow: [178.4155]

AMZN
today close [[1837.4268]] or [[1809.4828]] or [[1823.4548]](+/- 127)

GOOG
today close [[1131.2982]] or [[1132.281]] or [[1137.1511]](+/- 73)

NFLX
today close: 359.5414 (+/- 9.5)

ROKU
today close: [[97.16571]] (+/- 4.7)


S&P 500 (^GSPC)
TODAY CLOSE: [[2824.498]], CLOSE TODAY ACTUAL: 2826.06 
'''


'''try giving the open prices and make it predict the close, could be cool idk. like only using 
close as a label and nothing more, close wouldnt be in the data at all'''

print((prediction + prediction1)/2)

[[177.03635]]


[[179.37996]
 [178.4276 ]
 [170.96571]
 [175.49965]
 [171.13963]
 [166.89964]
 [167.77129]]
 
 prices for next 7 days
 we shall see
 appl